In [3]:
import tensorflow as tf  # Import TensorFlow library for deep learning
from tensorflow.keras.models import Sequential  # Sequential model for building neural networks layer by layer
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D  # Import necessary layers
from tensorflow.keras.callbacks import TensorBoard
import pickle  # Import the pickle library for loading data
import numpy as np  # Import NumPy for numerical operations
import time  # Import time for timestamp generation
import os  # Import os for handling file paths

# Create a unique name for the TensorBoard logs using the current timestamp
NAME = "Cats-vs-dog-cnn-64x2-{}".format(int(time.time()))

# Initialize TensorBoard callback with the correct log directory
log_dir = os.path.join("logs", NAME)
tensorboard = TensorBoard(log_dir=log_dir)

# Load preprocessed image data and labels from pickle files
X = pickle.load(open("X.pickle", "rb"))  # Load feature data (images)
y = pickle.load(open("y.pickle", "rb"))  # Load labels (categories)

# Normalize the pixel values of images to a range of 0 to 1
X = X / 255.0  

# Convert y to a NumPy array to ensure it's in the correct format for training
y = np.array(y)

# Verify that the number of samples in X matches the number of labels in y
assert X.shape[0] == y.shape[0], "Number of samples in X and y must match."

# Initialize a Sequential model
model = Sequential()

# Add convolutional layers and pooling layers
model.add(Conv2D(64, (3, 3), input_shape=X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output from the convolutional layers to feed into the dense layers
model.add(Flatten())

# Add a fully connected (dense) layer
model.add(Dense(64))
model.add(Activation("relu"))

# Output layer for binary classification
model.add(Dense(1))
model.add(Activation("sigmoid"))

# Compile the model with specified parameters
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

# Train the model using the training data and TensorBoard callback for logging
model.fit(X, y, batch_size=32, epochs=10, validation_split=0.3, callbacks=[tensorboard])

Epoch 1/10
1638/1638 [==============================] - 268s 163ms/step - loss: 0.5330 - accuracy: 0.7249 - val_loss: 0.4623 - val_accuracy: 0.7779
Epoch 2/10
1638/1638 [==============================] - 261s 160ms/step - loss: 0.3756 - accuracy: 0.8314 - val_loss: 0.3147 - val_accuracy: 0.8625
Epoch 3/10
1638/1638 [==============================] - 275s 168ms/step - loss: 0.2454 - accuracy: 0.8988 - val_loss: 0.1609 - val_accuracy: 0.9391
Epoch 4/10
1638/1638 [==============================] - 261s 160ms/step - loss: 0.1268 - accuracy: 0.9521 - val_loss: 0.0729 - val_accuracy: 0.9772
Epoch 5/10
1638/1638 [==============================] - 263s 161ms/step - loss: 0.0566 - accuracy: 0.9812 - val_loss: 0.0310 - val_accuracy: 0.9918
Epoch 6/10
1638/1638 [==============================] - 263s 161ms/step - loss: 0.0358 - accuracy: 0.9879 - val_loss: 0.0242 - val_accuracy: 0.9923
Epoch 7/10
1638/1638 [==============================] - 262s 160ms/step - loss: 0.0275 - accuracy: 0.9906 - val_